In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import conceptlab.api as cb

In [6]:
import anndata as ad
import scanpy as sc
import pandas as pd

In [7]:
!curl -o ../../data/demo/demo1.h5ad  https://datasets.cellxgene.cziscience.com/b4f8de4e-caf6-403c-a261-4e5298bb4707.h5ad

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  304M  100  304M    0     0  45.2M      0  0:00:06  0:00:06 --:--:-- 48.8M


In [7]:
adata = ad.read_h5ad('../../data/demo/demo1.h5ad')

In [8]:
adata.X = adata.X.todense()

/local_scratch/4932072/ipykernel_860482/3239206873.py:1: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  adata.X = adata.X.todense()


In [9]:
adata.obs['stimulation_status'] = pd.Categorical(adata.obs['stimulation_status'], categories = ['act', 'rest'])

In [10]:
cb.pp.concepts.curate_concepts(adata, concept_columns=['stimulation_status'])

Curated 1 concepts into adata.obsm['concepts']


In [11]:
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
subset_idx = np.random.choice(len(adata), size = 20_000)
train_idx, test_idx = train_test_split(subset_idx, test_size= 0.2)

In [13]:
adata_train = adata[train_idx].copy()
adata_test = adata[test_idx].copy()

/cv/scratch/u/andera29/micromamba/envs/conceptlab/lib/python3.11/site-packages/anndata/_core/anndata.py:1823: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [14]:
del adata

In [ ]:
model = cb.models.scCBGM()

In [ ]:
categorical_concept_mask = adata_train.uns['concepts_categorical_mask']

In [ ]:
model.fit(adata_train, concept_key = 'concepts', categorical_concept_mask= categorical_concept_mask, n_epochs=10)

In [22]:
output = model.encode(adata_test)

In [26]:
model.intervene(adata_test, new_concepts= np.random.randint(0,2, size = len(adata_test)).reshape(-1,1))

array([[-0.00027351, -0.00518859,  0.00316243, ...,  0.02193664,
         0.00172409,  0.00893657],
       [ 0.00589205, -0.00199187,  0.01080215, ...,  0.0231101 ,
         0.00037491, -0.00139502],
       [ 0.00430881, -0.00250424,  0.00356021, ...,  0.01923462,
        -0.00191053,  0.00196945],
       ...,
       [ 0.00769407, -0.00629643,  0.01702372, ...,  0.02701951,
        -0.00010454,  0.00308724],
       [ 0.00094703,  0.01237912,  0.01008767, ...,  0.02651522,
         0.00622059, -0.00770923],
       [ 0.00482834, -0.0075081 ,  0.00629825, ...,  0.01796862,
        -0.00197905,  0.00539469]], shape=(4000, 53792), dtype=float32)

In [27]:
model.decode(*output)

array([[-0.00284216, -0.00851078, -0.00650306, ...,  0.02152781,
         0.00730653,  0.01515725],
       [-0.00358832, -0.0051165 ,  0.00814874, ...,  0.02436688,
         0.00807474,  0.00176423],
       [ 0.00497803,  0.02598929,  0.01611359, ...,  0.01487114,
         0.00050204,  0.00346621],
       ...,
       [ 0.00679623, -0.01077207,  0.01721278, ...,  0.02071613,
        -0.00468278,  0.00540791],
       [ 0.00380312,  0.00951663,  0.01391274, ...,  0.02029258,
         0.00349281, -0.01193967],
       [ 0.00392899, -0.00578795,  0.00739008, ...,  0.02157767,
         0.00166598, -0.00018178]], shape=(4000, 53792), dtype=float32)